In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
api_url = "https://data.urbanaillinois.us/resource/afbd-8beq.csv"

The source link of the data: https://data.urbanaillinois.us/Police/Urbana-Police-Arrests-Since-1988/afbd-8beq

In [0]:
data = pd.read_csv(api_url, delimiter = ',')

In [4]:
data.head()

,arrest_code,incident_number,date_of_arrest,year_of_arrest,month_of_arrest,arrest_type_description,crime_code,crime_code_description,crime_category_code,crime_category_description,conspiracy_code,statute,violation,disposition_code,disposition_description,age_at_arrest,arrestee_sex,arrestee_race,arrestee_employment_description,arrestee_residency_description,arrestee_home_city,arrestee_home_state,arrestee_home_zip,arrest_resolution,mapped_address
0,A19-03536,U19-04248,2019-08-25T00:00:00.000,2019,8,SUMMONED,8701,NOISE PROHIBITION,C54,Noise & Vibrations Violations,NaN,UPD 16-2,NOISE PROHIBITION,88,SUMMONED / CITED / NOTIFIED - ADULT,20.0,FEMALE,BLACK,FULL-TIME STUDENT,RESIDENT,URBANA,IL,61801.0,NOTICE TO APPEAR - CITY,"\nURBANA, IL 61801\n(40.106957, -88.210905)"
1,A19-03538,U19-04261,2019-08-25T00:00:00.000,2019,8,ON VIEW,486,BATTERY-DOMESTIC,C05,Battery,NaN,720-5/12-3.2,BATTERY-DOMESTIC,87,ARRESTED AND HELD FOR PROSECUTION,39.0,MALE,BLACK,UNEMPLOYED,NON-RESIDENT,CHAMPAIGN,IL,61820.0,TAKEN TO JAIL,"\nCHAMPAIGN, IL 61820\n(40.112597, -88.244388)"
2,A19-03528,U19-04230,2019-08-24T00:00:00.000,2019,8,SUMMONED,460,BATTERY,C05,Battery,NaN,720-5/12-3,BATTERY,88,SUMMONED / CITED / NOTIFIED - ADULT,39.0,MALE,BLACK,UNKNOWN,RESIDENT,URBANA,IL,61801.0,NOTICE TO APPEAR - STATE,"\nURBANA, IL 61801\n(40.106957, -88.210905)"
3,A19-03532,U19-04233,2019-08-24T00:00:00.000,2019,8,ON VIEW,486,BATTERY-DOMESTIC,C05,Battery,NaN,720-5/12-3.2,BATTERY-DOMESTIC,87,ARRESTED AND HELD FOR PROSECUTION,19.0,FEMALE,ASIAN,NaN,NON-RESIDENT,CHAMPAIGN,IL,61820.0,TAKEN TO JAIL,"\nCHAMPAIGN, IL 61820\n(40.112597, -88.244388)"
4,A19-03534,U19-04246,2019-08-24T00:00:00.000,2019,8,ON VIEW,2430,ILLEGAL TRANSPORTATION OF LIQUOR,C27,Liquor Offenses,NaN,625-5/11-502,ILLEGAL TRANSPORTATION OF,88,SUMMONED / CITED / NOTIFIED - ADULT,36.0,MALE,HISPANIC,EMPLOYED,NON-RESIDENT,CHAMPAIGN,IL,61820.0,TAKEN TO JAIL,"\nCHAMPAIGN, IL 61820\n(40.112597, -88.244388)"


In [5]:
data.groupby(['arrestee_sex', 'arrestee_residency_description', 'arrestee_race', 'arrestee_employment_description', 'crime_category_description']).arrest_resolution.value_counts()

arrestee_sex  arrestee_residency_description  arrestee_race  arrestee_employment_description  crime_category_description     arrest_resolution       
FEMALE        NON-RESIDENT                    ASIAN          UNKNOWN                          Traffic Offenses               PROMISE TO COMPLY            7
                                                                                                                             INDIVIDUAL BOND              1
                                              BLACK          EMPLOYED                         Battery                        TAKEN TO JAIL                2
                                                                                              Burglary                       TAKEN TO JAIL                1
                                                                                              Controlled Substance Offenses  TAKEN TO JAIL                1
                                                                      

I chose 'arrestee_sex', 'arrestee_residency_description', 'arrestee_race', 'arrestee_employment_description', 'crime_category_description', and 'arrest_resolution' because they seems to tell a story when they are put together in a Sankey chart.

In [6]:
data.arrestee_sex.value_counts()

MALE      617
FEMALE    383
Name: arrestee_sex, dtype: int64

In [7]:
data.arrestee_race.value_counts()

BLACK       552
WHITE       357
HISPANIC     58
ASIAN        33
Name: arrestee_race, dtype: int64

In [8]:
data.arrestee_employment_description.value_counts()

UNKNOWN              626
UNEMPLOYED           189
EMPLOYED             137
FULL-TIME STUDENT     36
Name: arrestee_employment_description, dtype: int64

In [9]:
data.arrestee_residency_description.value_counts()

RESIDENT        466
NON-RESIDENT    460
UNKNOWN          69
Name: arrestee_residency_description, dtype: int64

In [10]:
data['arrestee_sex'].unique().tolist()

['FEMALE', 'MALE']

In [11]:
data.crime_category_description.value_counts()

Traffic Offenses                 528
Warrants & Summons                97
Theft                             72
Battery                           71
Interfering w/Public Officers     25
Controlled Substance Offenses     23
Weapons Offenses                  20
Trespassing                       19
Burglary                          15
Driving Under the Influence       12
Animal Offenses                   10
Drug Paraphernalia Act            10
Criminal Damage                    9
Disorderly Conduct                 9
Assault                            8
Robbery                            7
Motor Vehicle Theft                7
Offenses Involving Children        6
Domestic - Other Offenses          6
Liquor Offenses                    6
Kidnapping                         5
Cannabis Offenses                  4
Noise & Vibrations Violations      4
Deception & Fraud                  4
Criminal Sexual Assault            3
Status Offenses                    3
Violation Ord Protect/No Cont      2
B

Generally, the data is concise and clean.

In [12]:
import json as jn

json = dict()
json['nodes'] = list()
node = dict()
node['name'] = 'value'
json['nodes'].append(node)

json['links'] = list()
link = dict()
link['source'] = 0
link['dest'] = 1
link['value'] = 100
json['links'].append(link)

print(json)
print(jn.dumps(json))

{'nodes': [{'name': 'value'}], 'links': [{'source': 0, 'dest': 1, 'value': 100}]}
{"nodes": [{"name": "value"}], "links": [{"source": 0, "dest": 1, "value": 100}]}


Above is the requirerd data structure for D3.js Sankey chart. 

In [13]:
nodes = ['arrestee_sex', 'arrestee_residency_description', 'arrestee_race', 'arrestee_employment_description', 'crime_category_description', 'arrest_resolution']
index = dict()

json = dict()
json['nodes'] = list()

count = 0;
for node in nodes:
  for value in data[node].unique().tolist():
    json['nodes'].append({'name': value})
    index[value] = count
    count += 1
    
json['links'] = list()

    
print(jn.dumps(json))
print(index)

{"nodes": [{"name": "FEMALE"}, {"name": "MALE"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "BLACK"}, {"name": "ASIAN"}, {"name": "HISPANIC"}, {"name": "WHITE"}, {"name": "FULL-TIME STUDENT"}, {"name": "UNEMPLOYED"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "EMPLOYED"}, {"name": "Noise & Vibrations Violations"}, {"name": "Battery"}, {"name": "Liquor Offenses"}, {"name": "Driving Under the Influence"}, {"name": "Traffic Offenses"}, {"name": "Interfering w/Public Officers"}, {"name": "Weapons Offenses"}, {"name": "Theft"}, {"name": "Disorderly Conduct"}, {"name": "Controlled Substance Offenses"}, {"name": "Warrants & Summons"}, {"name": "Sex Offenses"}, {"name": "Prob/Parole/Bail Violations"}, {"name": "Assault"}, {"name": "Drug Paraphernalia Act"}, {"name": "Burglary"}, {"name": "Hypo Syringes & Needles Act"}, {"name": "Cannabis Offenses"}, {"name": "Kidnapping"}, {"name": "Criminal Damage"}, {"name": "Criminal Sexual Assault"}, {"na

'nodes' is a list of chosen columns.
'index' is used for looking up correct index value for source and dest in link creation.

Adding the nodes is trivial and the index dictionary seems to be okay. 
There are NAN in the data structures but it'll be ignored unless an issue occurs.

In [14]:
male = data[data['arrestee_sex'] == 'MALE']
female = data[data['arrestee_sex'] == 'FEMALE']
sex_res = male['arrestee_residency_description'].value_counts(dropna=False)

for sex in data['arrestee_sex'].unique().tolist():
  sex_data = data[data['arrestee_sex'] == sex]
  sex_res_data = sex_data['arrestee_residency_description'].value_counts(dropna=False)
  for key, val in sex_res_data.to_dict().items():
    link = {'source': index[sex], 'target': index[key], 'value': val}
    json['links'].append(link)


def create_links(source, dest):
  for sex in data[source].unique().tolist():
    sex_data = data[data[source] == sex]
    sex_res_data = sex_data[dest].value_counts(dropna=False)
    for key, val in sex_res_data.to_dict().items():
      link = {'source': index[sex], 'target': index[key], 'value': val}
      json['links'].append(link)
      
source = 'arrestee_residency_description'
dest = 'arrestee_race'
create_links(source, dest)

print(jn.dumps(json))

{"nodes": [{"name": "FEMALE"}, {"name": "MALE"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "BLACK"}, {"name": "ASIAN"}, {"name": "HISPANIC"}, {"name": "WHITE"}, {"name": "FULL-TIME STUDENT"}, {"name": "UNEMPLOYED"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "EMPLOYED"}, {"name": "Noise & Vibrations Violations"}, {"name": "Battery"}, {"name": "Liquor Offenses"}, {"name": "Driving Under the Influence"}, {"name": "Traffic Offenses"}, {"name": "Interfering w/Public Officers"}, {"name": "Weapons Offenses"}, {"name": "Theft"}, {"name": "Disorderly Conduct"}, {"name": "Controlled Substance Offenses"}, {"name": "Warrants & Summons"}, {"name": "Sex Offenses"}, {"name": "Prob/Parole/Bail Violations"}, {"name": "Assault"}, {"name": "Drug Paraphernalia Act"}, {"name": "Burglary"}, {"name": "Hypo Syringes & Needles Act"}, {"name": "Cannabis Offenses"}, {"name": "Kidnapping"}, {"name": "Criminal Damage"}, {"name": "Criminal Sexual Assault"}, {"na

Synthesis of the link creation function is broken into: trial by examples, using variables, writing a function.
create_links function takes in source and dest column labels and correctly creates the links.
There is no time for unit testing, the best testing will be the visualization itself.

In [15]:
from nltk.util import ngrams

bigram = list(ngrams(nodes, 2))
bigram

[('arrestee_sex', 'arrestee_residency_description'),
 ('arrestee_residency_description', 'arrestee_race'),
 ('arrestee_race', 'arrestee_employment_description'),
 ('arrestee_employment_description', 'crime_category_description'),
 ('crime_category_description', 'arrest_resolution')]

Using NLTK's ngram, the sources and targets are correctly organized for the create_links function.

> Indented block



In [16]:
nodes = ['arrestee_sex', 'arrestee_residency_description', 'arrestee_race', 'arrestee_employment_description', 'crime_category_description', 'arrest_resolution']
index = dict()

json = dict()
json['nodes'] = list()

count = 0;
for node in nodes:
  for value in data[node].unique().tolist():
    json['nodes'].append({'name': value})
    index[value] = count
    count += 1
    
json['links'] = list()

for gram in bigram:
  create_links(gram[0], gram[1])
  
  
print(jn.dumps(json))

{"nodes": [{"name": "FEMALE"}, {"name": "MALE"}, {"name": "RESIDENT"}, {"name": "NON-RESIDENT"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "BLACK"}, {"name": "ASIAN"}, {"name": "HISPANIC"}, {"name": "WHITE"}, {"name": "FULL-TIME STUDENT"}, {"name": "UNEMPLOYED"}, {"name": "UNKNOWN"}, {"name": NaN}, {"name": "EMPLOYED"}, {"name": "Noise & Vibrations Violations"}, {"name": "Battery"}, {"name": "Liquor Offenses"}, {"name": "Driving Under the Influence"}, {"name": "Traffic Offenses"}, {"name": "Interfering w/Public Officers"}, {"name": "Weapons Offenses"}, {"name": "Theft"}, {"name": "Disorderly Conduct"}, {"name": "Controlled Substance Offenses"}, {"name": "Warrants & Summons"}, {"name": "Sex Offenses"}, {"name": "Prob/Parole/Bail Violations"}, {"name": "Assault"}, {"name": "Drug Paraphernalia Act"}, {"name": "Burglary"}, {"name": "Hypo Syringes & Needles Act"}, {"name": "Cannabis Offenses"}, {"name": "Kidnapping"}, {"name": "Criminal Damage"}, {"name": "Criminal Sexual Assault"}, {"na

Now, the data is in correct format for Sankey diagram.

In [0]:
with open('data.json', 'w') as file:
    jn.dump(json, file)

Use Flow-o-matic: https://observablehq.com/@mbostock/flow-o-matic to save time.

JSON Online Viewer: http://jsonviewer.stack.hu/

D3.js Sankey: https://github.com/d3/d3-sankey